# "[NDIR] (KOR) Hard-negative Mining for Mir-Flickr 1M Dataset"
> NDIR tech review

- toc: false
- badges: false
- comments: false
- categories: [near-duplicate image detection, tech-review]
- hide_{github,colab,binder,deepnote}_badge: true

# 1. Overview
Hard-negative mining은 object detection task에서 자주 볼 수 있는 샘플링 방법중 하나이다. 여기서 말하는 Hard-negative란, negative(음성) 데이터중에 탐지되기 어려운 negative 데이터를 말하며, hard-negative mining은 이들을 우선적으로 뽑는 방법을 의미한다. NDIR 분야에서는 pair 단위로 ND라고 판단하기 어려운 데이터를 우선적으로 뽑는 방법을 의미한다. 대표적으로 다음 논문에서 사용했음을 확인할 수 있다.

> 2019, Expert Systems and Applications, Lia Morra, "Benchmarking Unsupervised Near-Duplicate Image Detection"

위의 논문에서는 Mir-Flickr 1M 이라는 데이터셋을 벤치마킹에 사용한다. 일반적으로 ND 문제를 계층적 군집화 방법으로 해결할 시, 굉장히 큰 공간복잡도를 필요로하고, 당연히 시간복잡도도 최소를 잡아도 $O(n^3)$이다. 현실적으로는 계산이 어려운 수치이고, 병렬화를 하더라도 한계가 있다. 우선 공간적인 문제를 해결하기 위해 군집화 문제를 이진 분류 문제로 치환하는 것이 가능하다. 만약 $n$개의 데이터가 있으면, 이를 $n \choose 2$개의 pair와 ND 여부를 label로 가질 수 있고, ND이면 1, NND이면 0을 가정하고 이진 분류가 가능하다. 이러한 방법은 다음 논문에서 사용했다.

> 2020, Mathematics, Zhili Zhou, "Near-Duplicate Image Detection System Using Coarse-to-Fine Matching Scheme Based on Global and Local CNN Features"

  이렇게 군집화 문제를 이진 분류 문제로 치환할 때 근본적으로 짚고 넘어가야할 문제가 있는데, pair elements간의 transivity 성질이다. 우선 ND로 판단된 클러스터가 존재한다면, 군집내의 모든 요소는 서로간에 transivity를 만족한다. 하지만, 이진 분류문제에서, $_{I_i} \text{ND}_{I_j} \,\ \text{and} \,\ _{I_j} \text{ND}_{I_k} \rightarrow _{I_i} \text{ND}_{I_k}$가 일반적으로 성립하지 않는다. (commutative는 성립한다.) 하지만, 조금더 일반적인 상황을 가정하기에 이는 바람직하다고 평가할 수는 있다. <br><br>
  
  이러한 방법들로 공간 복잡도를 해결했다면, 남은 것은 시간복잡도이다. MFND의 데이터 수는 10e6개이고 pair의 수는 4.999995e11개이다. 이전에 리뷰한 논문에서 이를 해결하는 방법을 담고있지만, Hard-negative mining을 이용하여 score를 근사하는 방식을 고려해보려한다.

# 2. Hard-negative mining for MFND dataset

- Randomly selected query images와 compact set of NND from a large image collection을 뽑는다. 이때, query image와 collection 내의 image에 대해서 near-duplicate match가 발생하지 않게 뽑는다.
    - EX) $1,000,000$개의 데이터에서 $K = 4400$개의 randomly selected query images와 $M = 80,000$개의 compact set을 뽑는다.
- 각 쿼리에 대해서 collection 내의 샘플에 대해서 거리를 계산할 수 있으며, 이는 $K \times M$ 행렬이된다.
- 각 행에 대해서 거리가 최소인 NND pair를 추출한다. ($hn_1$)
    - EX) $|hn_1| = 4400$
- 각 행을 정렬하여 query에 대해 K-nearest neighbors를 하여 추출한다. ($hn_2$)
    - EX) $|hn_2| = 4400 \times 5$
    
> $i$번째 쿼리 이미지와 first hard-negative sample간의 거리가 $j$번째 쿼리 이미지의 second hard-negative smaple간의 거리보다 클 수 있다. 그러므로 $hn_2$가 더 어려운 negative sample이라고 볼 수 있다.

# Commnets
- MFND와 같이 정말 큰 데이터셋을 동시병렬적으로 다루어야할 때, 좋은 방법인 것 같다. 이는 negative sample을 뽑는 방법이며, 분포를 잘 대표하는 샘플을 뽑는 것과 혼동하면 안될 것 같다.
- 간단하고 좋은 방법이지만, ND의 개념이 IND, NIND로 확장시킬 수 있다는 가정하에서는 좋지 않은 방법인 것 같다. 이러한 개념의 확장은 Duplicate~NND까지의 정도를 정량적으로 평가할 수 있는 새로운 개념이 정의될 수 있을 것인데, 위의 분석법을 사용하면, 모든 종류의 pair를 다룰 수 없는 상황이 발생할 것 같다.